# MPI messaging

This script will analyse `broadcast` and `barrier` messging, each with 3 different algorithms.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import plotly.express as px
import plotly.graph_objects as go

path = os.getcwd()

## `broadcast` messaging

Let's first load the data

In [17]:
import os
import pandas as pd
import plotly.graph_objects as go

# Set path to the data
bcast_path = path + '/bcast/results_bcast/'  # Replace with your actual path

# Get the list of files in the directory
files = os.listdir(bcast_path)

# Create a list to store the dataframes
dfs = []

# Read the data from the files; for each file, check if 
# the name contains binary, default or chain and set the
# corresponding value in the dataframe

for file in files:
    if 'binary' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Binary Tree'
        dfs.append(df)
    elif 'default' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Default'
        dfs.append(df)
    elif 'chain' in file:
        df = pd.read_csv(os.path.join(bcast_path, file))
        df['algorithm'] = 'Chain'
        dfs.append(df)

# Filter out dataframes where the 'Algorithm' column contains '${algorithm}' or is null
valid_dfs = []
for df in dfs:
    if 'Algorithm' in df.columns:
        df['Algorithm'] = df['Algorithm'].astype(str)
        if not df['Algorithm'].str.contains(r'\$\{algorithm\}').any():
            valid_dfs.append(df)

# Split the data into separate dataframes for each algorithm
default_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'Default']
binary_tree_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'Binary Tree']
chain_df = [df for df in valid_dfs if df['Algorithm'].iloc[0] == 'Chain']

fig = go.Figure()

# Function to add traces for each message size
def add_traces_for_algorithm(df_list, algorithm_name):
	for df in df_list:
		grouped = df.groupby('MessageSize')
		for message_size, group in grouped:
			fig.add_trace(go.Scatter(
					x=group['Processes'], 
					y=group['Latency'], 
					mode='lines+markers', 
					name=f'{algorithm_name} - {message_size} bytes'
				)
			)
	# Update layout
	fig.update_layout(
		title='Latency vs Processes for Different Algorithms',
		xaxis_title='Processes',
		yaxis_title='Latency (ms)',
		legend_title='Algorithm and Message Size',
		template='plotly'
	)
	fig.show()

# Add traces for each algorithm
add_traces_for_algorithm(default_df, 'default')
add_traces_for_algorithm(binary_tree_df, 'binary_tree')
add_traces_for_algorithm(chain_df, 'chain')




